<a href="https://colab.research.google.com/github/manjirisharma/Weather-Forecasting-System-using-ML/blob/main/Weather_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import libraries:**

In [11]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error,accuracy_score
from datetime import datetime, timedelta
import pytz

In [12]:
API_Key = '9873fba43a80c80f8e7ee1a023b22f19'
Base_URL = 'https://api.openweathermap.org/data/2.5/'

In [13]:
def get_current_weather(city):
  url = f"{Base_URL}weather?q={city}&appid={API_Key}&units=metric"
  response = requests.get(url)
  data = response.json()
  return {
      'city' : data['name'],
      'current_temp' : round(data['main']['temp']),
      'feels_like' : round(data['main']['feels_like']),
      'temp_min' : round(data['main']['temp_min']),
      'temp_max' : round(data['main']['temp_max']),
      'humidity' : data['main']['humidity'],
      'wind_speed' : data['wind']['speed'],
      'description' : data['weather'][0]['description'],
      'country' : data['sys']['country'],
      'wind_gust_dir' : data['wind'].get('deg', 0), # Use .get() with a default value
      'pressure' : data['main']['pressure'],
      'Wind_Gust_Speed' : data['wind'].get('gust', 0) # Use .get() with a default value
  }

In [14]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

In [15]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  X = data[['MinTemp','MaxTemp','WindGustSpeed','WindGustDir', 'Humidity', 'Pressure', 'Temp']]
  y = data['RainTomorrow']

  return X , y , le

In [16]:
def train_rain_model(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train,y_train)

  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)

  print(mean_squared_error(y_test, y_pred))
  return model


In [17]:
def prepare_regression_data(data, feature):
  X,y = [], []

  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1, 1)
  y = np.array(y)

  return X,y

In [18]:
def train_regression_model(X,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X,y)
  return model

In [19]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])

  return predictions[1:]

In [20]:
def weather_view():
  city = input('Enter any city name: ')
  current_weather = get_current_weather(city)

  historical_data = read_historical_data('weather.csv')

  X, y, le = prepare_data(historical_data)
  rain_model = train_rain_model(X,y)

  # map wind direction to compass points
  wind_dir = current_weather.get('wind_gust_dir', 0) % 360 # Use .get() with a default value
  compass_points = [('N',0,11.25), ('NNE',11.25,33.75), ('NE',33.75,56.25), ('ENE',56.25,78.75), ('E',78.75,101.25), ('ESE',101.25,123.75), ('SE',123.75,146.25), ('SSE',146.25,168.75), ('S',168.75,191.25), ('SSW',191.25,213.75), ('SW',213.75,236.25), ('WSW',236.25,258.75), ('W',258.75,281.25), ('WNW',281.25,303.75), ('NW',303.75,326.25), ('NNW',326.25,348.75)]

  compass_direction = next((point for point, start, end in compass_points if start <= wind_dir < end), 'N') # Provide a default value if no match

  # Handle unseen wind directions during prediction
  if compass_direction not in le.classes_:
    le.classes_ = np.append(le.classes_, compass_direction)

  compass_direction_encoded = le.transform([compass_direction])[0]


  current_data = {
      'MinTemp' : current_weather['temp_min'],
      'MaxTemp' : current_weather['temp_max'],
      'WindGustDir' : compass_direction_encoded,
      'WindGustSpeed' : current_weather.get('Wind_Gust_Speed', 0), # Use .get() with a default value
      'Humidity' : current_weather['humidity'],
      'Pressure' : current_weather['pressure'],
      'Temp' : current_weather['current_temp']
  }

  # Ensure the order of columns matches the training data
  feature_names = ['MinTemp','MaxTemp','WindGustSpeed','WindGustDir', 'Humidity', 'Pressure', 'Temp']
  current_data_df = pd.DataFrame([current_data], columns=feature_names)

  #rain prediction

  rain_prediction = rain_model.predict(current_data_df)[0]

  # prepare regression model for temperature and humidity

  X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(X_temp, y_temp)
  hum_model = train_regression_model(X_hum, y_hum)

  # predict future temperature and humidity

  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_humidity = predict_future(hum_model, current_weather['humidity'])

  #prepare time for future predictions

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  # Display results

  print(f"City : {city}, {current_weather['country']}")
  print(f"Current Temperature : {current_weather['current_temp']}°C")
  print(f"Feels Like : {current_weather['feels_like']}°C")
  print(f"Minimum Temperature : {current_weather['temp_min']}°C")
  print(f"Maximum Temperature : {current_weather['temp_max']}°C")
  print(f"Humidity : {current_weather['humidity']}%")
  print(f"Wind Speed : {current_weather['wind_speed']}m/s")
  print(f"Description : {current_weather['description']}")
  print(f"Rain Predictions : {'Yes' if rain_prediction == 1 else 'No'}")

  print("\nFuture Temperatures :")
  for time, temp in zip(future_times, future_temp):
    print(f"{time} : {round(temp, 1)}°C")

  print("\nFuture Humidity :")
  for time, humidity in zip(future_times, future_humidity):
    print(f"{time} : {round(humidity, 1)}%")

weather_view()

Enter any city name: delhi
Accuracy: 0.821917808219178
0.1780821917808219
City : delhi, IN
Current Temperature : 30°C
Feels Like : 29°C
Minimum Temperature : 30°C
Maximum Temperature : 30°C
Humidity : 28%
Wind Speed : 4.12m/s
Description : haze
Rain Predictions : No

Future Temperatures :
15:00 : 28.5°C
16:00 : 22.0°C
17:00 : 23.9°C
18:00 : 21.4°C
19:00 : 22.7°C

Future Humidity :
15:00 : 27.9%
16:00 : 27.9%
17:00 : 27.9%
18:00 : 27.9%
19:00 : 27.9%
